### Imports

In [1]:
import h5py
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import csv


from tqdm import tqdm
from pathlib import Path
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler

In [2]:
BASEDIR = Path("")

fn = BASEDIR / "train_eval.hdf5"

### Pre Evaluation

Iterate over the different Groups and summarize them to get an idea of which to use to analyze

In [3]:
objectWithMask = 0
objectWithPost_fire = 0
objectWithPre_fire = 0

objectsSummary = []
currentObject = {}

def listObject(obj, indent=0):
    global objectWithMask
    global objectWithPost_fire
    global objectWithPre_fire
    global objectsSummary
    global currentObject

    
    
    for name, thing in obj.items():
        if indent == 0:
            currentObject = { "name" : "", "mask": False, "pre_fire" : False, "post_fire": False}

        if isinstance(thing, h5py.Group):
            if indent == 0:
                currentObject["name"] = name
            listObject(thing, indent+1)
            
        else:
            if name == "mask":
                currentObject["mask"] = True
                objectWithMask += 1
            elif name == "post_fire":
                currentObject["post_fire"] = True
                objectWithPost_fire += 1
            elif name == "pre_fire":
                currentObject["pre_fire"] = True
                objectWithPre_fire += 1
            else:
                print(name)
    
        if indent == 0:
            objectsSummary.append(currentObject)

In [4]:
with h5py.File(fn, "r") as fd:
    listObject(fd)

In [5]:
print("Items with mask: " + str(objectWithMask))
print("Items with pre_fire: " + str(objectWithPre_fire))
print("Items with post_fire: " + str(objectWithPost_fire))
print("Overall Items: " + str(len(objectsSummary)))

Items with mask: 534
Items with pre_fire: 356
Items with post_fire: 534
Overall Items: 534


### Analysis Functions

Xarray Transformation

In [6]:
# Function copied from practice
def to_xarray(dataset, pretty_band_names=True):
    """... Ergänzen Sie hier eine guten Docstring ... """
    
    if pretty_band_names:
        BANDS = ["coastal_aerosol", "blue", "green", "red",
                 "veg_red_1", "veg_red_2", "veg_red_3", "nir", 
                 "veg_red_4", "water_vapour", "swir_1", "swir_2"]
    else:
        BANDS = ["1", "2", "3", "4", "5", "6", "7", "8", "8a", "9", "11", "12"]
        
    post = dataset["post_fire"][...].astype("float32") / 10000.0
    
    # Da `pre_fire` manchmal fehlt ersetzen wir es durch 0 Werte was
    # eh der Platzhalter für einen fehlenden Messwert ist.
    try:
        pre = dataset["pre_fire"][...].astype("float32") / 10000.0
    except KeyError:
        pre = np.zeros_like(post, dtype="float32")
        
    # Da die Maske nur ein "Band" hat können wir die dritte Dimension einfach
    # weglassen. Das erreichen wir in dem wir mit `0` am Ende indizieren.
    mask = dataset["mask"][..., 0]
    
    return {"pre": xr.DataArray(pre, dims=["x", "y", "band"], coords={"x": range(512), "y": range(512), "band": BANDS}),
            "post": xr.DataArray(post, dims=["x", "y", "band"], coords={"x": range(512), "y": range(512), "band": BANDS}),
            "mask": xr.DataArray(mask, dims=["x", "y"], coords={"x": range(512), "y": range(512)}),
            "fold": dataset.attrs["fold"]}

In [7]:
def scoreBands(xdDataArray, xdMask):
    bands = []
    scaler = MinMaxScaler()
    flatxdMask = xdMask.values.flatten()

    for band in xdDataArray.band:
        bandScore = {}
        bandScore["name"] = band.values

        dataPoints = 0
        truePoints = 0
        falsePoints = 0
        overallScore = 0.0
        falseScore = 0.0
        trueScore = 0.0
        minTrue = 0.0
        maxTrue = 0.0
        minFalse = 0.0
        maxFalse = 0.0


        selBand = xdDataArray.sel(band=band).values
        normband = scaler.fit_transform(selBand)
        flatBand = normband.flatten()
        
        for i, k in enumerate(flatxdMask):
            dataPoints += 1
            point = flatBand[i]
            if k:
                truePoints += 1
                if (minTrue > point):
                    minTrue = point

                if (maxTrue < point):
                    maxTrue = point

                trueScore += point
            else:
                falsePoints += 1
                if (minFalse > point):
                    minFalse = point
                    
                if (maxFalse < point):
                    maxFalse = point

                falseScore += point

        overallScore = trueScore + falseScore
        bandScore["dataPoints"] = dataPoints
        bandScore["truePoints"] = truePoints
        bandScore["falsePoints"] = falsePoints
        bandScore["overallScore"] = overallScore
        bandScore["trueScore"] = trueScore
        bandScore["falseScore"] = falseScore

        if dataPoints != 0:
            bandScore["avgScore"] = overallScore/dataPoints
        else:
            bandScore["avgScore"] = 0
        
        if truePoints != 0:
            bandScore["avgTrue"] = trueScore/truePoints
        else :
            bandScore["avgTrue"] = trueScore

        if falsePoints != 0:
            bandScore["avgFalse"] = falseScore/falsePoints
        else:
            bandScore["avgFalse"] = falseScore
            
        bandScore["minTrue"] = minTrue
        bandScore["maxTrue"] = maxTrue
        bandScore["minFalse"] = minFalse
        bandScore["maxFalse"] = maxFalse
        

        bands.append(bandScore)
    return bands

### Single Case Evaluation

Visualizing some of the images provided: 
Looking at pre- and post- image

In [8]:
def showComposedPrePostImage(id):
    with h5py.File(fn, "r") as fd:
        data = to_xarray(fd[id])

    pre = data["pre"]
    post = data["post"]

    # Gamma Korrektur **0.45
    fig, ax =  plt.subplots(nrows=2, ncols=1, figsize=(12, 3*12))

    ax[0].imshow(pre.sel(band=["red", "green", "blue"]).values**0.45, vmin=0, vmax=1, interpolation="nearest")
    ax[1].imshow(post.sel(band=["red", "green", "blue"]).values**0.45, vmin=0, vmax=1, interpolation="nearest")
    # ax[1].imshow(deltaMask.sel(band=["red", "green", "blue"]).values**0.45, vmin=0, vmax=1, interpolation="nearest")
    for i in range(2):
        ax[i].axis("off")
    fig.tight_layout()

In [9]:
#showComposedPrePostImage("0328d12a-4ad8-4504-8ac5-70089db10b4e_0")

Comparing single bands using images

In [10]:
def showImageBandsComparison(id):
    with h5py.File(fn, "r") as fd:
        data = to_xarray(fd[id])

    pre = data["pre"]
    post = data["post"]
    mask = data["mask"]
    diff = data["post"] - data["pre"]

    n_bands = len(pre.band)
    fig, ax =  plt.subplots(nrows=n_bands, ncols=4, figsize=(4 * 4, 4 * n_bands))

    for i, band in enumerate(pre.band):
        ax[i, 0].imshow(diff.sel(band=band).values, cmap='gray', interpolation="nearest")
        ax[i, 0].set_title(band.values)
        ax[i, 1].imshow(pre.sel(band=band).values, cmap='gray', interpolation="nearest")
        ax[i, 1].set_title("pre")
        ax[i, 2].imshow(post.sel(band=band).values, cmap='gray', interpolation="nearest")
        ax[i, 2].set_title("post")
        ax[i, 3].imshow(mask.values, interpolation="nearest")
        ax[i, 3].set_title("mask")
        for j in range(4):
            ax[i, j].axis("off")
    fig.tight_layout()

In [11]:
#showImageBandsComparison("0328d12a-4ad8-4504-8ac5-70089db10b4e_0")

Display dispersion of bands in affected area

In [12]:
# Calculate the delta between the pre and post-fire images
def showDispersonByAffected(id):
    
    with h5py.File(fn, "r") as fd:
        data = to_xarray(fd[id])

    pre = data["pre"]
    post = data["post"]
    mask = data["mask"]
    diff = data["post"] - data["pre"]
    
    flatMask = mask.values.flatten()


    fig, ax = plt.subplots(figsize=(16, 8))
    for band in post.band:
        selBand = diff.sel(band=band).values.flatten()
        dataPoints = []

        for i, k in enumerate(flatMask):
            if k:
                dataPoints.append(selBand[i])

        ax.plot(dataPoints, label=band.values)

    plt.legend()
    plt.show()

In [13]:
#showDispersonByAffected("0328d12a-4ad8-4504-8ac5-70089db10b4e_0")

Display dispersion of bands outside affected Area

In [14]:
def showDispersionByUnaffected(id):
    with h5py.File(fn, "r") as fd:
        data = to_xarray(fd[id])

    pre = data["pre"]
    post = data["post"]
    mask = data["mask"]
    diff = data["post"] - data["pre"]

  
    flatMask = mask.values.flatten()

    fig, ax = plt.subplots(figsize=(16, 8))
    for band in post.band:
        selBand = diff.sel(band=band).values.flatten()
        dataPoints = []

        for i, k in enumerate(flatMask):
            if not k:
                dataPoints.append(selBand[i])

        ax.plot(dataPoints[::800], label=band.values)

    plt.legend()
    plt.show()

In [15]:
#showDispersionByUnaffected("0328d12a-4ad8-4504-8ac5-70089db10b4e_0")

Distribution of Bands overall

In [16]:
def showOverallDistributionOfBands(id):
    with h5py.File(fn, "r") as fd:
        data = to_xarray(fd[id])

    post = data["post"]
    diff = data["post"] - data["pre"]

    scaler = MinMaxScaler()

    fig, ax = plt.subplots(figsize=(16, 8))
    for band in post.band:
        selBand = diff.sel(band=band).values
        normband = scaler.fit_transform(selBand)
        flatBand = normband.flatten()

        sns.kdeplot(flatBand, label = band.values)

    ax.set_xlabel('Delta')
    ax.set_ylabel('Density')
    plt.legend()
    plt.show()

In [17]:
#showOverallDistributionOfBands("0328d12a-4ad8-4504-8ac5-70089db10b4e_0")

Distribution of Bands that are in affected areas

In [18]:
def showOverallDistributionOfBandsByAffected(id):
    with h5py.File(fn, "r") as fd:
        data = to_xarray(fd[id])

    pre = data["pre"]
    post = data["post"]
    mask = data["mask"]
    diff = data["post"] - data["pre"]

    flatMask = mask.values.flatten()
    scaler = MinMaxScaler()

    fig, ax = plt.subplots(figsize=(16, 8))
    for band in post.band:
        selBand = diff.sel(band=band).values
        normband = scaler.fit_transform(selBand)
        flatBand = normband.flatten()
        
        #dataPoints = delta.sel(band=band).values
        dataPoints = []

        for i, k in enumerate(flatMask):
            if k:
                dataPoints.append(flatBand[i])
        
        #ax.plot(dataPoints, label=band.values)
        sns.kdeplot(dataPoints, label = band.values)

    ax.set_xlabel('Delta')
    ax.set_ylabel('Density')
    plt.legend()
    plt.show()

In [19]:
#showOverallDistributionOfBandsByAffected("0328d12a-4ad8-4504-8ac5-70089db10b4e_0")

Visualizing how each band scores

In [20]:
def showBandScoresDistribution(id):
    with h5py.File(fn, "r") as fd:
        data = to_xarray(fd[id])

    pre = data["pre"]
    post = data["post"]
    mask = data["mask"]

    bands = scoreBands(post, mask)

    xLabel = "trueScore"
    yLabel = "falseScore"

    fig, ax = plt.subplots(figsize=(16, 8))
    for band in bands:
        
        truthiness = band["trueScore"] / band["falseScore"]
        
        ax.scatter(x = band[xLabel], 
                y = band[yLabel], 
                s = (truthiness * 20000), 
                label=band["name"])

    ax.set_xlabel(xLabel)
    ax.set_ylabel(yLabel)
    plt.legend()
    plt.show()

In [21]:
#showBandScoresDistribution("0328d12a-4ad8-4504-8ac5-70089db10b4e_0")

Visualizing how accurate each band is

In [22]:
def showBandScoresAccuracy(id):
    with h5py.File(fn, "r") as fd:
        data = to_xarray(fd[id])

    pre = data["pre"]
    post = data["post"]
    mask = data["mask"]

    fig, ax = plt.subplots(figsize=(16, 8))
    bands = scoreBands(post, mask)
    for band in bands:
        

        trueVariation = abs(band["maxTrue"] - band["minTrue"])
        

        
        ax.scatter(x = band["avgFalse"], 
                y = band["avgTrue"], 
                s = (1 + trueVariation) ** 6, 
                label=band["name"])

    ax.set_xlabel("avgFalse")
    ax.set_ylabel("avgTrue")
    ax.grid(True)
    plt.legend()
    plt.show()

In [23]:
#showBandScoresAccuracy("0328d12a-4ad8-4504-8ac5-70089db10b4e_0")

### Broad Analysis

In [24]:
globalScores = { "entries" : [], "count" : 0 }

In [25]:
def readAllData():
    with h5py.File(fn, "r") as fd:
        for entry in objectsSummary:
            if entry["mask"] and entry["pre_fire"] and entry["post_fire"]:
                globalEntry = { "descriptor" : entry, "scores" : {} }
                globalScores["count"] += 1
                xdData = to_xarray(fd[entry["name"]])

                prexdData = xdData["pre"]
                postxdData = xdData["post"]
                deltaxdData = postxdData - prexdData
                maskxdData = xdData["mask"]

                globalEntry["scores"]["preScores"] = scoreBands(prexdData, maskxdData)
                globalEntry["scores"]["postScores"] = scoreBands(postxdData, maskxdData)
                globalEntry["scores"]["deltaScores"] = scoreBands(deltaxdData, maskxdData)

                globalScores["entries"].append(globalEntry)


In [26]:
def writeTocsv(globalScores):
    scoreTypes = ["preScores", "postScores", "deltaScores"]
    scoreKeys = ["truePoints", "falsePoints", "overallScore", "trueScore", "falseScore", "avgScore", "avgTrue", "avgFalse", "minTrue", "maxTrue", "minFalse", "maxFalse" ]

    with open('scores.csv', 'w', newline='') as csvfile:
        scoresWriter = csv.writer(csvfile, delimiter=';',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        #write header
        scoresWriter.writerow(["id", "dataType", "band"] + scoreKeys)

        #write data
        for entry in globalScores["entries"]:
            for scoreType in scoreTypes:
                for bandScore in entry["scores"][scoreType]:
                    data = []

                    for scoreKey in scoreKeys:
                        data.append(bandScore[scoreKey])

                    scoresWriter.writerow([
                        entry["descriptor"]["name"],
                        scoreType,
                        bandScore["name"]
                    ] + data)

In [48]:
def readFromCsv():
    scoreTypes = ["preScores", "postScores", "deltaScores"]
    scoreKeys = ["truePoints", "falsePoints", "overallScore", "trueScore", "falseScore", "avgScore", "avgTrue", "avgFalse", "minTrue", "maxTrue", "minFalse", "maxFalse" ]

    scoreFileData = { "entries" : [] , "count" : 0}
    with open('scores.csv', 'r', newline='') as csvfile:
        scoresReader = csv.reader(csvfile, delimiter=';',
                                quotechar='|')
        
        count = 0
        entry = { "descriptor" : { "name" : "", "mask": True, "pre_fire" : True, "post_fire": True}, "scores" : {}}
        for row in scoresReader:
            
            if count == 0:
                count += 1
                continue
            
            key = ""
            band = {}
            for i, value in enumerate(row):
                if i == 0:
                    if entry["descriptor"]["name"] != value:
                        if entry["descriptor"]["name"] != "":
                            scoreFileData["entries"].append(entry)
                        entry = { "descriptor" : { "name" : value, "mask": True, "pre_fire" : True, "post_fire": True}, "scores" : {}}
                        count += 1

                elif i == 1:
                    key = value
                    if not key in entry["scores"]:
                        entry["scores"][key] = []
                elif i == 2:
                    band["name"] = value
                else:
                    band[scoreKeys[i - 3]] = float(value)

            entry["scores"][key].append(band)

        scoreFileData["count"] = count

    return scoreFileData

In [49]:
#globalScores = readAllData()
globalScores = readFromCsv()

In [50]:
globalAverage = {}

In [51]:
def generateAverage(globalScores): 
    keys = ["preScores", "postScores", "deltaScores"]
    for entry in globalScores["entries"]:
        for key in keys:
            if not key in globalAverage:
                globalAverage[key] = { "truePoints": 0.0, "falsePoints": 0.0, "overallScore": 0.0, "trueScore": 0.0, "falseScore": 0.0, 
                                    "avgScore": 0.0, "avgTrue": 0.0, "avgFalse": 0.0, "minTrue": 0.0, "maxTrue": 0.0, "minFalse": 0.0, "maxFalse": 0.0 }


            for bandScore in entry["scores"][key]:

                globalAverage[key]["truePoints"] += bandScore["truePoints"]
                globalAverage[key]["falsePoints"] +=bandScore["falsePoints"]
                globalAverage[key]["overallScore"] +=bandScore["overallScore"]
                globalAverage[key]["trueScore"] += bandScore["trueScore"]
                globalAverage[key]["falseScore"] += bandScore["falseScore"]
                globalAverage[key]["avgScore"] += bandScore["avgScore"]
                globalAverage[key]["avgTrue"] += bandScore["avgTrue"]
                globalAverage[key]["avgFalse"] += bandScore["avgFalse"]

                if not globalAverage[key]["minTrue"] or globalAverage[key]["minTrue"] > bandScore["minTrue"]:
                    globalAverage[key]["minTrue"] = bandScore["minTrue"]

                if not globalAverage[key]["maxTrue"] or globalAverage[key]["maxTrue"] < bandScore["maxTrue"]:
                    globalAverage[key]["maxTrue"] = bandScore["maxTrue"]

                if not globalAverage[key]["minFalse"] or globalAverage[key]["minFalse"] > bandScore["minFalse"]:
                    globalAverage[key]["minFalse"] = bandScore["minFalse"]

                if not globalAverage[key]["maxFalse"] or globalAverage[key]["maxFalse"] < bandScore["maxFalse"]:
                    globalAverage[key]["maxFalse"] = bandScore["maxFalse"]

    for key in keys:  
        globalAverage[key]["truePoints"] = globalAverage[key]["truePoints"] / globalScores["count"]
        globalAverage[key]["falsePoints"] = globalAverage[key]["falsePoints"] / globalScores["count"]
        globalAverage[key]["overallScore"] = globalAverage[key]["overallScore"] / globalScores["count"]
        globalAverage[key]["trueScore"] = globalAverage[key]["trueScore"] / globalScores["count"]
        globalAverage[key]["falseScore"] = globalAverage[key]["falseScore"] / globalScores["count"]
        globalAverage[key]["avgScore"] = globalAverage[key]["avgScore"] / globalScores["count"]
        globalAverage[key]["avgTrue"] = globalAverage[key]["avgTrue"] / globalScores["count"]
        globalAverage[key]["avgFalse"] = globalAverage[key]["avgFalse"] / globalScores["count"]

In [52]:
globalAverage = generateAverage(globalScores=globalScores)